In [ ]:
import os
import sys

import numpy as np
import pandas as pd

from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.models.widgets import RangeSlider

import msprime

sys.path.append("../src")
import util
import vis

output_notebook()


In [ ]:
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = '0aaf0agotd3etfja916liv2etcl4ul9j3fk8kav1m1a16m18da6b'


In [ ]:
ts = msprime.sim_mutations(
    msprime.sim_ancestry(
        100,
        sequence_length=1e7,
        ploidy=2,
        recombination_rate=1e-8,
        random_seed=1234
    ),
    rate=1e-7,
    random_seed=1234
)
ts


In [ ]:
path = util.SamplePath(
    individual="test",
    nodes=np.concatenate((
        np.repeat( 80, 5),
        np.repeat(100, 6),
        np.repeat(240, 6),
        np.repeat(350, 5),
        np.repeat( 70, 4),
        np.repeat(260, 5),
    )),
    site_positions=ts.sites_position,
)
path


In [ ]:
np.random.seed(1234)
fwd_prob_mat_ar = np.random.uniform(size=(ts.num_nodes, ts.num_sites))
fwd_prob_mat_df = pd.DataFrame({
    'node_id' : np.array([np.repeat(i, ts.num_sites) for i in np.arange(ts.num_nodes)]).flatten(),
    'site_id' : np.array([np.arange(ts.num_sites) for _ in np.arange(ts.num_nodes)]).flatten(),
    'site_pos': np.tile(ts.sites_position, ts.num_nodes).flatten(),
    'prob'    : fwd_prob_mat_ar.flatten(),
})
fwd_prob_mat_df


In [ ]:
# TODO: Use toggle buttons for the different views.
controls = {
    "interval": RangeSlider(
        start=0, end=len(path), step=1,
        value=(0, len(path)),
        title="Genomic interval"
    )
}

tracks_site_pos = [
    path.site_positions,
    path.site_positions,
    path.site_positions,
    path.site_positions,
]
tracks_site_id = [
    np.arange(len(path)),
    np.arange(len(path)),
    np.arange(len(path)),
    np.arange(len(path)),
]

app = vis.create_step_chart_app(
    path,
    ts,
    markers=path.site_positions[::2],
    tracks=tracks_site_pos,
    legend_labels=[
        'BEAGLE',
        'tskit',
        'truth',
        'chip',
    ],
    colors=[
        'blue',
        'orange',
        'green',
        'grey',
    ],
    #matrix=fwd_prob_mat_df,
    controls=controls,
)

show(app)
